In [28]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium.webdriver.chrome.service import Service
import time
import datetime
import pandas as pd
import lxml
import re
import random
from IPython.display import display, Markdown

In [29]:
# get data from existing file
example_df = pd.read_csv("./05-07-25.csv")

# identify missing problems
last_val = example_df.tail(1).id

last_val

missing_problems = set([i for i in range(1, last_val.iloc[0] + 1)]) ^ set([example_df.iloc[i].iloc[0] for i in range(0, len(example_df))])
missing_problems

{84,
 85,
 86,
 87,
 88,
 89,
 116,
 206,
 237,
 267,
 359,
 448,
 449,
 479,
 480,
 569,
 600,
 630,
 691,
 724,
 814,
 845}

In [30]:
columns = ["id", "title", "subtitle", "content", "html_content", "release_date", "solved_by_count", "difficulty"]

def getMissingProblems(problems = [], df=pd.DataFrame(columns=columns)):
    if(len(problems) == 0): return None
    
    # curr_problem = problems[0]
    # url = "https://projecteuler.net/problem={}".format(curr_problem)

    # executable_path = {'executable_path': 'chromedriver.exe'}
    # browser = Browser('chrome', **executable_path, headless=False)
    my_service = Service(executable_path='./chromedriver.exe')
    browser = Browser('chrome', service=my_service, headless=False)

    browser.visit("https://projecteuler.net/")

    #add a delay so page fully loads
    # time.sleep(8)

    # problems = []
    # df = pd.DataFrame(columns=columns)
    count = 0

    while(count < len(problems)):
        curr_problem = problems[count]
        url = "https://projecteuler.net/problem={}".format(curr_problem)
        browser.visit(url)
        time.sleep(8 + 4 * random.random())
        
        print(curr_problem)
        current_problem = browser.html
        
        try:
            news_soup = bs(current_problem, "html.parser")

            title = news_soup.find_all("h2")[0].get_text()
            subtitle = news_soup.find_all("h3")[0].get_text()
            content = news_soup.find_all("div", class_="problem_content")[0].get_text()
            tooltip = news_soup.find_all(class_="tooltiptext_right")[0].get_text()

            # split tooltip text into separate features
            # datetime (may need to set UTC as timezone? Or a different timezone?)
            datetime_str = re.findall("Published on (.*[ap]m);", tooltip)
    #         print(datetime_str)
            datetime_str = re.sub("(?<=\d)[a-z][a-z]", "", datetime_str[0])
            datetime_str = re.sub("(?<= )(\d)(?= )", "0\g<0>", datetime_str)
            datetime_str = datetime_str.replace("pm", "PM")
            datetime_str = datetime_str.replace("am", "AM")
    #         print(datetime_str)
            datetime_val = datetime.datetime.strptime(datetime_str, "%A, %d %B %Y, %I:%M %p")

            # Solved By (count)
            solved_by_count = re.findall("Solved by (\d+)", tooltip)

            # Difficulty Rating
            difficulty = re.findall("(\d+)%", tooltip)
            if(len(difficulty) == 0):
                difficulty = [float('nan')]
                
            # get problem html content
            html_url = "https://projecteuler.net/minimal={}".format(curr_problem)
            browser.visit(html_url)
            
            time.sleep(1.37)
            curr_html = bs(browser.html, "html.parser")
            
            html_content = curr_html.find_all("pre")[0].get_text()
            # make img tags use absolute paths
            html_content = re.sub(r'<img src="(?!http)', '<img src="https://projecteuler.net/', html_content)
            # make anchor tags use absolute paths
            html_content = re.sub(r'<a href="(?!http)', '<a href="https://projecteuler.net/', html_content)
            print(html_content)

            curr_row = pd.Series({"id":curr_problem, "title":title, "subtitle":subtitle, "content":content, "html_content":html_content, "release_date":datetime_val.isoformat(), "solved_by_count":int(solved_by_count[0]), "difficulty":int(difficulty[0])})
            df = pd.concat([df, curr_row.to_frame().T], ignore_index=True)
#         print(curr_row)
        except:
            pass
        count += 1

    # song to know when scraping done
    browser.visit("https://www.youtube.com/watch?v=kln_bIndDJg")
    
    return df


<>:44: SyntaxWarning: invalid escape sequence '\d'
<>:45: SyntaxWarning: invalid escape sequence '\d'
<>:45: SyntaxWarning: invalid escape sequence '\g'
<>:52: SyntaxWarning: invalid escape sequence '\d'
<>:55: SyntaxWarning: invalid escape sequence '\d'
<>:44: SyntaxWarning: invalid escape sequence '\d'
<>:45: SyntaxWarning: invalid escape sequence '\d'
<>:45: SyntaxWarning: invalid escape sequence '\g'
<>:52: SyntaxWarning: invalid escape sequence '\d'
<>:55: SyntaxWarning: invalid escape sequence '\d'
C:\Users\PGendotti\AppData\Local\Temp\ipykernel_75932\2020711415.py:44: SyntaxWarning: invalid escape sequence '\d'
  datetime_str = re.sub("(?<=\d)[a-z][a-z]", "", datetime_str[0])
C:\Users\PGendotti\AppData\Local\Temp\ipykernel_75932\2020711415.py:45: SyntaxWarning: invalid escape sequence '\d'
  datetime_str = re.sub("(?<= )(\d)(?= )", "0\g<0>", datetime_str)
C:\Users\PGendotti\AppData\Local\Temp\ipykernel_75932\2020711415.py:45: SyntaxWarning: invalid escape sequence '\g'
  datetim

In [ ]:
test_df = getMissingProblems(list(missing_problems))
test_df

In [40]:
# merge with existing df
for i in range(len(test_df)):
    curr_row = test_df.iloc[i]
    
    example_df = pd.concat([example_df, curr_row.to_frame().T], ignore_index=True)

example_df.drop_duplicates(inplace=True)
example_df.sort_values(by="id", inplace=True)
example_df.set_index(keys="id", inplace=True)

# then sort
print(len(example_df))

example_df


933


,title,subtitle,content,html_content,release_date,solved_by_count,difficulty
id,,,,,,,
1,Multiples of 3 or 5,Problem 1,\nIf we list all the natural numbers below 10 ...,<p>If we list all the natural numbers below $1...,2001-10-05T18:00:00,1027961,5
2,Even Fibonacci Numbers,Problem 2,\nEach new term in the Fibonacci sequence is g...,<p>Each new term in the Fibonacci sequence is ...,2001-10-19T18:00:00,818478,5
3,Largest Prime Factor,Problem 3,"\nThe prime factors of 13195 are 5,7,13 and 29...","<p>The prime factors of $13195$ are $5, 7, 13$...",2001-11-02T18:00:00,589607,5
4,Largest Palindrome Product,Problem 4,\nA palindromic number reads the same both way...,<p>A palindromic number reads the same both wa...,2001-11-16T18:00:00,520937,5
5,Smallest Multiple,Problem 5,\n2520 is the smallest number that can be divi...,<p>$2520$ is the smallest number that can be d...,2001-11-30T18:00:00,523353,5
...,...,...,...,...,...,...,...
929,Odd-Run Compositions,Problem 929,\nA composition of n is a sequence of positive...,<p>A <b>composition</b> of $n$ is a sequence o...,2025-01-26T07:00:00,210,35
930,The Gathering,Problem 930,"\nGiven n≥2 bowls arranged in a circle, m≥2 ba...","<p>Given $n\ge 2$ bowls arranged in a circle, ...",2025-02-02T10:00:00,115,75
931,Totient Graph,Problem 931,\n\nFor a positive integer n construct a graph...,<p>\nFor a positive integer $n$ construct a gr...,2025-02-08T13:00:00,137,55


403
<p>
For integers <var>a</var> and <var>b</var>, we define <var>D</var>(<var>a</var>, <var>b</var>) as the domain enclosed by the parabola <var>y</var> = <var>x</var><sup>2</sup> and the line <var>y</var> = <var>a</var>·<var>x</var> + <var>b</var>:<br /><var>D</var>(<var>a</var>, <var>b</var>) = { (<var>x</var>, <var>y</var>) | <var>x</var><sup>2</sup> ≤ <var>y</var> ≤ <var>a</var>·<var>x</var> + <var>b</var> }.
</p>
<p>
L(<var>a</var>, <var>b</var>) is defined as the number of lattice points contained in <var>D</var>(<var>a</var>, <var>b</var>).<br />
For example, L(1, 2) = 8 and L(2, -1) = 1.
</p>
<p>
We also define S(<var>N</var>) as the sum of L(<var>a</var>, <var>b</var>) for all the pairs (<var>a</var>, <var>b</var>) such that the area of <var>D</var>(<var>a</var>, <var>b</var>) is a rational number and |<var>a</var>|,|<var>b</var>| ≤ <var>N</var>.<br />
We can verify that S(5) = 344 and S(100) = 26709528.
</p>
<p>
Find S(10<sup>12</sup>). Give your answer mod 10<sup>8</sup>.


411
<p>
Let <var>n</var> be a positive integer. Suppose there are stations at the coordinates (<var>x</var>, <var>y</var>) = (2<sup><var>i</var></sup> mod <var>n</var>, 3<sup><var>i</var></sup> mod <var>n</var>) for 0 ≤ <var>i</var> ≤ 2<var>n</var>. We will consider stations with the same coordinates as the same station.
</p><p>
We wish to form a path from (0, 0) to (<var>n</var>, <var>n</var>) such that the x and y coordinates never decrease.<br />
Let S(<var>n</var>) be the maximum number of stations such a path can pass through.
</p><p>
For example, if <var>n</var> = 22, there are 11 distinct stations, and a valid path can pass through at most 5 stations. Therefore, S(22) = 5.
The case is illustrated below, with an example of an optimal path:
</p>
<p align="center"><img src="https://projecteuler.net/project/images/p411_longpath.png" alt="p411_longpath.png" /></p>
<p>
It can also be verified that S(123) = 14 and S(10000) = 48.
</p><p>
Find <span style="font-size:larger;"><span style=

418
<p>
Let <var>n</var> be a positive integer. An integer triple (<var>a</var>, <var>b</var>, <var>c</var>) is called a <i>factorisation triple</i> of <var>n</var> if:</p><ul><li> 1 ≤ <var>a</var> ≤ <var>b</var> ≤ <var>c</var>
</li><li> <var>a</var>·<var>b</var>·<var>c</var> = <var>n</var>.
</li></ul><p>
Define <var>f</var>(<var>n</var>) to be <var>a</var> + <var>b</var> + <var>c</var> for the factorisation triple (<var>a</var>, <var>b</var>, <var>c</var>) of <var>n</var> which minimises <var>c</var> / <var>a</var>. One can show that this triple is unique.
</p>
<p>
For example, <var>f</var>(165) = 19, <var>f</var>(100100) = 142 and <var>f</var>(20!) = 4034872.
</p>
<p>
Find <var>f</var>(43!).
</p>

419
<p>
The <b>look and say</b> sequence goes 1, 11, 21, 1211, 111221, 312211, 13112221, 1113213211, ...<br />
The sequence starts with 1 and all other members are obtained by describing the previous member in terms of consecutive digits.<br />
It helps to do this out loud:<br />
1 is 'one 

...

795
<p>
For a positive integer $n$, the function $g(n)$ is defined as
</p>
$$\displaystyle g(n)=\sum_{i=1}^{n} (-1)^i \gcd \left(n,i^2\right)$$
<p>
For example, $g(4) = -\gcd \left(4,1^2\right) + \gcd \left(4,2^2\right) - \gcd \left(4,3^2\right) + \gcd \left(4,4^2\right) = -1+4-1+4=6$.<br />
You are also given $g(1234)=1233$.
</p>
<p>
Let $\displaystyle G(N) = \sum_{n=1}^N g(n)$. You are given $G(1234) = 2194708$.
</p>
<p>
Find $G(12345678)$.
</p>

796
<p>A standard $52$ card deck comprises thirteen ranks in four suits. However, modern decks have two additional Jokers, which neither have a suit nor a rank, for a total of $54$ cards. If we shuffle such a deck and draw cards without replacement, then we would need, on average, approximately $29.05361725$ cards so that we have at least one card for each rank.</p>

<p>Now, assume you have $10$ such decks, each with a different back design. We shuffle all $10 \times 54$ cards together and draw cards without replacement. What is the expected

807
<p>Given a circle $C$ and an integer $n &gt; 1$, we perform the following operations.</p>

<p>In step $0$, we choose two uniformly random points $R_0$ and $B_0$ on $C$.<br />
In step $i$ ($1 \leq i &lt; n$), we first choose a uniformly random point $R_i$ on $C$ and connect the points $R_{i - 1}$ and $R_i$ with a red rope; then choose a uniformly random point $B_i$ on $C$ and connect the points $B_{i - 1}$ and $B_i$ with a blue rope.<br />
In step $n$, we first connect the points $R_{n - 1}$ and $R_0$ with a red rope; then connect the points $B_{n - 1}$ and $B_0$ with a blue rope.<br />
Each rope is straight between its two end points, and lies above all previous ropes.</p>

<p>After step $n$, we get a loop of red ropes, and a loop of blue ropes.<br />
Sometimes the two loops can be separated, as in the left figure below; sometimes they are "linked", hence cannot be separated, as in the middle and right figures below.</p>

<div style="text-align:center;">
<img src="https://projecteu

820
<p>Let $d_n(x)$ be the $n$<sup>th</sup> decimal digit of the fractional part of $x$, or $0$ if the fractional part has fewer than $n$ digits.</p>
<p>For example:</p>
<ul>
<li>$d_7 \mathopen{}\left( 1 \right)\mathclose{} = d_7 \mathopen{}\left( \frac 1 2 \right)\mathclose{} = d_7 \mathopen{}\left( \frac 1 4 \right)\mathclose{} = d_7 \mathopen{}\left( \frac 1 5 \right)\mathclose{} = 0$</li>
<li>$d_7 \mathopen{}\left( \frac 1 3 \right)\mathclose{} = 3$ since $\frac 1 3 =$ 0.333333<span style="color:#FF0000;font-weight:bold;">3</span>333...</li>
<li>$d_7 \mathopen{}\left( \frac 1 6 \right)\mathclose{} = 6$ since $\frac 1 6 =$ 0.166666<span style="color:#FF0000;font-weight:bold;">6</span>666...</li>
<li>$d_7 \mathopen{}\left( \frac 1 7 \right)\mathclose{} = 1$ since $\frac 1 7 =$ 0.142857<span style="color:#FF0000;font-weight:bold;">1</span>428...</li>
</ul>
<p>Let $\displaystyle  S(n) = \sum_{k=1}^n d_n \mathopen{}\left( \frac 1 k \right)\mathclose{}$.</p>
<p>You are given:</p>
<ul>
<l

830
<p>
Let $\displaystyle S(n)=\sum\limits_{k=0}^{n}\binom{n}{k}k^n$.</p>

<p>
You are given, $S(10)=142469423360$.</p>

<p>
Find $S(10^{18})$. Submit your answer modulo $83^3 89^3 97^3$.</p>


831
<p>Let $g(m)$ be the integer defined by the following double sum of products of binomial coefficients:</p>
<p>
$$\sum_{j=0}^m\sum_{i = 0}^j (-1)^{j-i}\binom mj \binom ji \binom{j+5+6i}{j+5} $$
</p>
<p>
You are given that $g(10) = 127278262644918$.<br /> Its first (most significant) five digits are $12727$.<br />

Find the first ten digits of $g(142857)$ when written in base 7.
</p>

832
<p>
In this problem $\oplus$ is used to represent the bitwise <strong>exclusive or</strong> of two numbers.<br />
Starting with blank paper repeatedly do the following:</p>

<ol type="1">
<li>Write down the smallest positive integer $a$ which is currently not on the paper;</li>

<li>Find the smallest positive integer $b$ such that neither $b$ nor $(a \oplus b)$ is currently on the paper. Then write down both

,id,title,subtitle,content,html_content,release_date,solved_by_count,difficulty
0,1,Multiples of 3 or 5,Problem 1,\nIf we list all the natural numbers below 10 ...,<p>If we list all the natural numbers below 10...,2001-10-05T18:00:00,970054,5
1,2,Even Fibonacci numbers,Problem 2,\nEach new term in the Fibonacci sequence is g...,<p>Each new term in the Fibonacci sequence is ...,2001-10-19T18:00:00,774340,5
2,3,Largest prime factor,Problem 3,"\nThe prime factors of 13195 are 5, 7, 13 and ...","<p>The prime factors of 13195 are 5, 7, 13 and...",2001-11-02T18:00:00,557877,5
3,4,Largest palindrome product,Problem 4,\nA palindromic number reads the same both way...,\n<p>A palindromic number reads the same both ...,2001-11-16T18:00:00,493994,5
4,5,Smallest multiple,Problem 5,\n2520 is the smallest number that can be divi...,<p>2520 is the smallest number that can be div...,2001-11-30T18:00:00,497316,5
...,...,...,...,...,...,...,...,...
825,826,Birds on a Wire,Problem 826,\nConsider a wire of length 1 unit between two...,<p>Consider a wire of length 1 unit between tw...,2023-01-22T10:00:00,185,45
826,827,Pythagorean Triple Occurrence,Problem 827,\n\nDefine Q(n) to be the smallest number that...,<p>\nDefine $Q(n)$ to be the smallest number t...,2023-01-28T13:00:00,147,50
827,828,Numbers Challenge,Problem 828,\nIt is a common recreational problem to make ...,<p>It is a common recreational problem to make...,2023-02-04T16:00:00,401,15
828,829,Integral Fusion,Problem 829,\nGiven any integer n>1 a binary factor tree T...,<p>Given any integer $n&gt;1$ a <i>binary fact...,2023-02-11T19:00:00,173,45


In [42]:
display(Markdown(example_df["html_content"][313]))

<p>In a sliding game a counter may slide horizontally or vertically into an empty space. The objective of the game is to move the red counter from the top left corner of a grid to the bottom right corner; the space always starts in the bottom right corner. For example, the following sequence of pictures show how the game can be completed in five moves on a $2$ by $2$ grid.</p>

<div align="center"><img src="https://projecteuler.net/resources/images/0313_sliding_game_1.gif?1678992056" alt="0313_sliding_game_1.gif"></div>

<p>Let $S(m,n)$ represent the minimum number of moves to complete the game on an $m$ by $n$ grid. For example, it can be verified that $S(5,4) = 25$.</p>

<div align="center"><img src="https://projecteuler.net/resources/images/0313_sliding_game_2.gif?1678992056" alt="0313_sliding_game_2.gif"></div>

<p>There are exactly $5482$ grids for which $S(m,n) = p^2$, where $p \lt 100$ is prime.</p>

<p>How many grids does $S(m,n) = p^2$, where $p \lt 10^6$ is prime?</p>


In [43]:
curr_date = datetime.datetime.now().strftime("%m-%d-%y")
example_df.to_csv(f"./{curr_date}-patched.csv")

# Outline of PE Scraping Process

- navigate to PE problem 1 (https://projecteuler.net/problem=1)
- scrape webpage from browser using beautifulsoup
- add problem information to dataframe
- increment problem number
- repeat navigation, scrape, and data append for each problem until returned to the main archives page (https://projecteuler.net/archives)
- output data to a file (.csv)
- should take ~1 hour for 800 problems (given a 5 second page load time)

## To Do
- ~~Fix problem descriptions that use MathJax (https://docs.mathjax.org/en/v1.1-latest/mathjax.html) notation (highest priority because it directly affects problem description clarity and readability, which directly impacts the dataset's quality as a problem repository)~~
- Add data files for problems that use external files (like Poker Hands)
  - include images as well as problem data files
  - currently uses links, make a separate save folder
- ~~Add links for problems that use external information (like Roman Numerals)~~
- Add a notebook to update the problem set file (only add new problems since older problems do not change often)
- Can also rescrape entire notebook since number of solvers for a problem should increase over time? (may want to add versioning using a date/timestamp, or can just use git)